In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

# Set image size and paths (same as your provided structure)
img_height, img_width = 100, 100
batch_size = 32
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Data generator for training, validation, and testing
datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(train_dir,
                                         target_size=(img_height, img_width),
                                         batch_size=batch_size,
                                         class_mode='categorical')

val_data = datagen.flow_from_directory(val_dir,
                                       target_size=(img_height, img_width),
                                       batch_size=batch_size,
                                       class_mode='categorical')

test_data = datagen.flow_from_directory(test_dir,
                                        target_size=(img_height, img_width),
                                        batch_size=batch_size,
                                        class_mode='categorical')

# Load MobileNetV2 pre-trained model, excluding the top (classification) layers
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False, weights='imagenet')

# Freeze all layers initially
base_model.trainable = False

# Create a new model by adding custom layers on top of MobileNetV2
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Pool the features to reduce dimensionality
    Dense(128, activation='relu'),  # Add a fully connected layer
    Dense(train_data.num_classes, activation='softmax')  # Final layer with the number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model on your dataset
model.fit(train_data, validation_data=val_data, epochs=20, callbacks=[early_stopping])

# Unfreeze the last 50 layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-50]:  # Freeze all layers except the last 50 layers
    layer.trainable = False

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
model.fit(train_data, validation_data=val_data, epochs=10, callbacks=[early_stopping])

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data)
print(f'Test Accuracy: {test_accuracy}')

# Save the fine-tuned model
model.save('face_40_transfer_learning.keras')


Found 168 images belonging to 6 classes.
Found 36 images belonging to 6 classes.
Found 36 images belonging to 6 classes.


C:\Users\user\AppData\Local\Temp\ipykernel_18996\1395044948.py:33: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 7s 1us/step
Epoch 1/20


C:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - accuracy: 0.3398 - loss: 1.9384 - val_accuracy: 0.9722 - val_loss: 0.2281
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 832ms/step - accuracy: 0.9460 - loss: 0.1686 - val_accuracy: 1.0000 - val_loss: 0.0319
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 729ms/step - accuracy: 0.9983 - loss: 0.0163 - val_accuracy: 1.0000 - val_loss: 0.0181
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 949ms/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 1.0000 - val_loss: 0.0168
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 722ms/step - accuracy: 1.0000 - loss: 0.0067 - val_accuracy: 1.0000 - val_loss: 0.0107
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 16s 755ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 1.0000 - val_loss: 0.0072
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 703ms/step - accuracy: 1.0000 - loss: 8.8885e-04 - val_accuracy: 1.0000 - val_loss: 0.0052
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 697ms/step - accuracy: 1.0000 - loss: 7.6132e-04 - val_accuracy: 1.0000 - val_los